### In this notebook, we will import the Groove Midi Dataset and process it, and save it for easily being used in our pipelines


In [1]:
import tensorflow as tf
tf.enable_eager_execution() 

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# For some reason, tfds import gives an error on the first attempt but works second time around
try:                              
    import tensorflow_datasets as tfds
except:
    import tensorflow_datasets as tfds
    
# Import necessary libraries for processing/loading/storing the dataset
import numpy as np
import pickle
import pandas as pd
import ipyparams
from shutil import copy2

# Import libraries for creating/naming folders/files
import os, sys
from datetime import datetime

<IPython.core.display.Javascript object>

In [3]:
# Import the HVO_Sequence implementation
sys.path.append("../") # --> unnecessary in future implementations (when package installed via pip)
from hvo_sequence.io_helpers import note_sequence_to_hvo_sequence
from hvo_sequence.drum_mappings import ROLAND_REDUCED_MAPPING

# Import magenta's note_seq 
import note_seq

In [11]:


# tfds works in both Eager and Graph modes
#tf.enable_eager_execution() #not needed in TF V2, as it is already the default
#import magenta.music as mm 
# from magenta.models.music_vae import data   <---- DELETE
#import magenta

#print (magenta.__version__)
#from visual_midi import Plotter
#from pretty_midi import PrettyMIDI
#from IPython.core.display import display, HTML


#### We have the dataset already available in tensorflow_datasets, so let's load it first


#####  NOTE:
 you can convert midi to note_sequence, two ways:
 1. USE magenta.music.midi_to_note_sequence, BUT THIS will GIVE YOU A PICKLING ERROR
 2. install note_seq (pip install note_seq) then use note_seq.midi_to_note_sequence

#### Let's Start With the 2bar-midionly dataset

In [4]:
dataset_train_unprocessed,dataset_train_info = tfds.load(
    name="groove/2bar-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True,
    with_info=True)

dataset_test_unprocessed = tfds.load(
    name="groove/2bar-midionly",
    split=tfds.Split.TEST,
    try_gcs=True)

dataset_validation_unprocessed = tfds.load(
    name="groove/2bar-midionly",
    split=tfds.Split.VALIDATION,
    try_gcs=True)

### NOTE: TF DATASETS ARE PYTHON ITERABLES
you can convert the dataset into a list or numpy array:

   * List(dataset)
   * tfds.as_numpy(dataset)
    
Lets see how many samples we've got in the dataset



In [5]:
 # In all three sets, separate entries into individual examples 
dataset_train = dataset_train_unprocessed.batch(1)
dataset_test  = dataset_test_unprocessed.batch(1)
dataset_validation = dataset_validation_unprocessed.batch(1)

print("\n Number of Examples in Train Set: {}, Test Set: {}, Validation Set: {}".format(
    len(list(dataset_train)), 
    len(list(dataset_test)), 
    len(list(dataset_validation)))
     ) 


 Number of Examples in Train Set: 18163, Test Set: 2204, Validation Set: 2252


In [6]:
# Download groovemidi set (midi files and metadata of performances, not chopped files!) 
!mkdir ../resources
!mkdir ../resources/source_dataset
!source activate torch_thesis; wget -N -P ../resources/source_dataset https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip
!unzip -n ../resources/source_dataset/groove-v1.0.0-midionly.zip -d ../resources/source_dataset

mkdir: ../resources: File exists
mkdir: ../resources/source_dataset: File exists
--2021-04-27 20:07:42--  https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip
Resolving storage.googleapis.com... 142.250.200.144, 142.250.184.16, 216.58.215.176, ...
Connecting to storage.googleapis.com|142.250.200.144|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File '../resources/source_dataset/groove-v1.0.0-midionly.zip' not modified on server. Omitting download.

Archive:  ../resources/source_dataset/groove-v1.0.0-midionly.zip


### Now, we will start pre-process the downloaded 2-bar groove midi datasets.
The metadata for the dataset is not available via the tfds.load() method. As a result, I have manually downloaded the groove-v1.0.0-midionly.zip dataset from here:
        
        https://magenta.tensorflow.org/datasets/groove
        
In this folder (currently in resources/source_dataset), there is a csv file containing the metadata for each of the examples. I will match each of the 2-bar examples (downloaded from tfds.load()) with the csv file available in the manually downloaded groove-v1.0.0-midionly.zip


To keep track of the metadata, a panda's dataframe with the following fields is used

* Keys from Groove MIDI Dataset:
    * bpm
    * drummer
    * id 
    * midi
    * style
    * time_signature
    * type

In [7]:
dataframe = pd.read_csv('../resources/source_dataset/groove/info.csv', delimiter = ',')
dataframe.head()

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split
0,drummer1,drummer1/eval_session,drummer1/eval_session/1,funk/groove1,138,beat,4-4,drummer1/eval_session/1_funk-groove1_138_beat_...,drummer1/eval_session/1_funk-groove1_138_beat_...,27.872308,test
1,drummer1,drummer1/eval_session,drummer1/eval_session/10,soul/groove10,102,beat,4-4,drummer1/eval_session/10_soul-groove10_102_bea...,drummer1/eval_session/10_soul-groove10_102_bea...,37.691158,test
2,drummer1,drummer1/eval_session,drummer1/eval_session/2,funk/groove2,105,beat,4-4,drummer1/eval_session/2_funk-groove2_105_beat_...,drummer1/eval_session/2_funk-groove2_105_beat_...,36.351218,test
3,drummer1,drummer1/eval_session,drummer1/eval_session/3,soul/groove3,86,beat,4-4,drummer1/eval_session/3_soul-groove3_86_beat_4...,drummer1/eval_session/3_soul-groove3_86_beat_4...,44.716543,test
4,drummer1,drummer1/eval_session,drummer1/eval_session/4,soul/groove4,80,beat,4-4,drummer1/eval_session/4_soul-groove4_80_beat_4...,drummer1/eval_session/4_soul-groove4_80_beat_4...,47.987500,test


In [8]:
# A quick guide to using the panda dataframe
(dataframe[dataframe.id == "drummer1/eval_session/1"]["session"]).to_numpy()

array(['drummer1/eval_session'], dtype=object)

# Difference between tfds.load(name="groove/2bar-midionly") and groove-v1.0.0-midionly.zip

In groove-v1.0.0-midionly.zip, we have access to full performances, while using tfds.load(name="groove/2bar-midionly"), we can readily access the performance chopped into 2 bar segments. 

However, in the pre-chopped set, the meta data is missing. 

Hence, we need to find the relevant metadata in the info.csv file available groove-v1.0.0-midionly.zip. 

To do so, we match the beginning of the id in the chopped segments with the id in info.csv


# Preprocessing Steps


To preprocess the set, we will go through each of the examples in train/test/eval sets and do the following:

    1. get the midi file from the "midi" field available from tfds.load()
    2. convert midi to note_sequence 
    2. convert the note_sequence to HVO_Sequence
    3. get the id from the "id" field available from tfds.load()
    4. Match the beginning of the "id" field with the ids in the pandas dataframe (loaded from groove-v1.0.0-midionly.zip)
    5. Store all processed/retrieved fields in a dictionary
    

In [9]:
def dict_append(dictionary, key, vals):
    # Appends a value or a list of values to a key in a dictionary
    
    # if the values for a key are not a list, they are converted to a list and then extended with vals
    dictionary[key]=list(dictionary[key]) if not isinstance(dictionary[key], list) else dictionary[key]
    
    # if vals is a single value (not a list), it's converted to a list so as to be iterable
    vals = [vals] if not isinstance(vals, list) else vals
        
    # append new values 
    for val in vals:
        dictionary[key].append(val)

    return dictionary

def convert_groove_midi_dataset(dataset, beat_division_factors=[4], csv_dataframe_info=None):
    
    dataset_dict_processed = dict()
    dataset_dict_processed.update({
        "drummer":[],
        "session":[],
        "loop_id":[],  # the id of the recording from which the loop is extracted
        "master_id":[], # the id of the recording from which the loop is extracted
        "style_primary":[],
        "style_secondary":[],
        "bpm":[],
        "beat_type":[],
        "time_signature":[],
        "full_midi_filename":[],
        "full_audio_filename":[],
        "midi":[],
        "note_sequence":[],
        "hvo_sequence":[],
    })
    
    for features in dataset:
        
        # Features to be extracted from the dataset
        
        note_sequence = note_seq.midi_to_note_sequence(tfds.as_numpy(features["midi"][0]))
        
        
        if note_sequence.notes: # ignore if no notes in note_sequence (i.e. empty 2 bar sequence)
                        
            _hvo_seq = note_sequence_to_hvo_sequence(
                ns = note_sequence, 
                drum_mapping = ROLAND_REDUCED_MAPPING,
                beat_division_factors = beat_division_factors
            )
            
            if not csv_dataframe_info.empty:

                # Master ID for the Loop
                main_id = features["id"].numpy()[0].decode("utf-8").split(":")[0]

                # Get the relevant series from the dataframe
                df = csv_dataframe_info[csv_dataframe_info.id == main_id]
                
                # Update the dictionary associated with the metadata
                dict_append(dataset_dict_processed, "drummer", df["drummer"].to_numpy()[0])
                dict_append(dataset_dict_processed, "session", df["session"].to_numpy()[0].split("/")[-1])
                dict_append(dataset_dict_processed, "loop_id", features["id"].numpy()[0].decode("utf-8"))
                dict_append(dataset_dict_processed, "master_id", main_id)

                style_full = df["style"].to_numpy()[0]
                style_primary = style_full.split("/")[0]
                dict_append(dataset_dict_processed, "style_primary", style_primary)

                if "/" in style_full:
                    style_secondary = style_full.split("/")[1]
                    dict_append(dataset_dict_processed, "style_secondary", style_secondary)
                else:
                    dict_append(dataset_dict_processed, "style_secondary", ["None"])

                dict_append(dataset_dict_processed, "bpm", df["bpm"].to_numpy()[0])
                dict_append(dataset_dict_processed, "beat_type", df["beat_type"].to_numpy()[0])
                dict_append(dataset_dict_processed, "time_signature", df["time_signature"].to_numpy()[0])
                dict_append(dataset_dict_processed, "full_midi_filename", df["midi_filename"].to_numpy()[0])
                dict_append(dataset_dict_processed, "full_audio_filename", df["audio_filename"].to_numpy()[0])

                dict_append(dataset_dict_processed, "midi", features["midi"].numpy()[0])
                dict_append(dataset_dict_processed, "note_sequence", [note_sequence])
                        
            dict_append(dataset_dict_processed, "hvo_sequence", _hvo_seq)

        else:
            pass 
            
    return dataset_dict_processed

In [10]:
%%time

# Process Training Set
dataset_train = dataset_train_unprocessed.batch(1)
dataset_train_processed = convert_groove_midi_dataset(
    dataset = dataset_train, 
    beat_division_factors=[4], 
    csv_dataframe_info=dataframe
)

# Process Test Set
dataset_test = dataset_test_unprocessed.batch(1)
dataset_test_processed = convert_groove_midi_dataset(
    dataset = dataset_test, 
    beat_division_factors=[4], 
    csv_dataframe_info=dataframe)

# Process Validation Set
dataset_validation = dataset_validation_unprocessed.batch(1)
dataset_validation_processed = convert_groove_midi_dataset(
    dataset = dataset_validation, 
    beat_division_factors=[4], 
    csv_dataframe_info=dataframe)

CPU times: user 1min 10s, sys: 2.13 s, total: 1min 12s
Wall time: 1min 11s


### Order the pre-processed set
The batching process shuffles the dataset. However, we prefer to store the dataset in sequential order. Hence, we need to manually order the dataset. In this tutorial, I order the sequences by loop_id.

In [11]:
def sort_dictionary_by_key (dictionary_to_sort, key_used_to_sort):
    # sorts a dictionary according to the list within a given key
    sorted_ids=np.argsort(dictionary_to_sort[key_used_to_sort])
    for key in dictionary_to_sort.keys():
        dictionary_to_sort[key]=[dictionary_to_sort[key][i] for i in sorted_ids]
    return dictionary_to_sort

In [12]:
# Sort the sets using ids
dataset_train_processed = sort_dictionary_by_key(dataset_train_processed, "loop_id")
dataset_test_processed = sort_dictionary_by_key(dataset_test_processed, "loop_id")
dataset_validation_processed = sort_dictionary_by_key(dataset_validation_processed, "loop_id")

--------------------------------------
--------------------------------------

# Storing Pre-Processed Sets

The folder structure is as follows:

../processed_dataset

      |
      |--> /Processed_On_%d_%m_%Y_at_%H_%M_hrs
            |
            |--> /GrooveMIDI_processed_{train, test, or validation}
                    |
                    |--> hvo_sequence.obj
                    |
                    |--> note_sequence.obj
                    |
                    |--> midi.obj
                    |
                    |--> midi.obj
                    |
                    |--> metadata.csv
                    
                    
The midi files, hvo_sequences and note_sequences along with corresponding metadata are all saved in separate files. The order of entries in all these files match one another (increasing in alpha-numeric order of "loopid"s)

In [13]:
# DUMP INTO A PICKLE FILE
def store_dataset_as_pickle(dataset_list, 
                            filename_list, 
                            root_dir = "../processed_dataset",
                            append_datetime=True, 
                            features_with_separate_picklefile = ["hvo", "midi", "note_seq"]
                           ):

    #filename = filename.split(".obj")[0]
    path = root_dir
    
    if append_datetime:
        now = datetime.now()
        dt_string = now.strftime("%d_%m_%Y_at_%H_%M_hrs")
    else:
        dt_string =""
        
    path = os.path.join(path, "Processed_On_"+dt_string)
    
    if not os.path.exists (path):
        os.makedirs(path)

    currentNotebook = ipyparams.notebook_name
    print("Copying Source Code from %s to %s" % (os.path.join(os.getcwd(), currentNotebook), path))
    print("-"*100)

    # copy2(os.path.join(os.getcwd(), currentNotebook), path) 
    
    for i, dataset in enumerate(dataset_list):

        subdirectory = os.path.join(path, filename_list[i])
        if not os.path.exists (subdirectory):
            os.makedirs(subdirectory)
            
        print("-"*100)
        print("-"*100)
        print("Processing %s folder" % subdirectory)
        print("-"*100)
        print("-"*100)
        
        # Create Metadata File
        csv_dataframe = pd.DataFrame()
        
        for k in dataset.keys():
            if k not in features_with_separate_picklefile:
                csv_dataframe[k] = dataset[k]
        
        csv_dataframe.to_csv(os.path.join(subdirectory, "metadata.csv"))
        
        print("Metadata created!")
        print("-"*100)

        for feature in features_with_separate_picklefile:
            if feature in dataset.keys():
                dataset_filehandler = open(os.path.join(subdirectory, "%s_data.obj"%feature),"wb")
                print(feature)
                print(dataset_filehandler)
                pickle.dump(dataset[feature],  dataset_filehandler)
                dataset_filehandler.close()
                print("feature %s pickled at %s" % (feature, os.path.join(subdirectory, "%s.obj"%filename_list[i].split(".")[0])))
                print("-"*100)

            else:
                 raise Warning("Feature is not available: ", feature)


In [14]:
dataset_list = [dataset_train_processed,
               dataset_test_processed,
               dataset_validation_processed]

filename_list = ["GrooveMIDI_processed_train",
                "GrooveMIDI_processed_test",
                "GrooveMIDI_processed_validation"]

store_dataset_as_pickle(dataset_list, 
                        filename_list,
                        root_dir="../processed_dataset",
                        append_datetime=True,
                        features_with_separate_picklefile = ["hvo_sequence", "midi", "note_sequence"]
                       )

Copying Source Code from /Users/pelinski/tpr/smc/tfm/code/GrooveMIDI2HVO_PreProcessing/pre_processing_tutorial/GrooveMIDI2HVO-Tutorial.ipynb to ../processed_dataset/Processed_On_27_04_2021_at_20_09_hrs
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Processing ../processed_dataset/Processed_On_27_04_2021_at_20_09_hrs/GrooveMIDI_processed_train folder
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Metadata created!
----------------------------------------------------------------------------------------------------
hvo_sequence
<_io.BufferedWriter name='../processed_dataset/Processed_O

---------
----------

### Using Pre-Processed Sets After Storing

Now, we can import the pre-processed sets easily into our scripts without going through this procedure. 

To do so, load the pickle files and the metadata from corresponding subfolders!


In [16]:
source_path = "../processed_dataset/Processed_On_27_04_2021_at_19_04_hrs"
print(os.path.join(source_path, "GrooveMIDI_processed_train", "hvo_data.obj"))
train_file = open(os.path.join(source_path, "GrooveMIDI_processed_train", "hvo_sequence_data.obj"),'rb')
train_set = pickle.load(train_file)
metadata = pd.read_csv(os.path.join(source_path, "GrooveMIDI_processed_train", "metadata.csv"))

features_in_metadata = list(metadata.columns)

dataset_size = len(train_set)

print("Dataset Size: %d \n Features: " % dataset_size, features_in_metadata)

../processed_dataset/Processed_On_27_04_2021_at_19_04_hrs/GrooveMIDI_processed_train/hvo_data.obj
Dataset Size: 17108 
 Features:  ['Unnamed: 0', 'drummer', 'session', 'loop_id', 'master_id', 'style_primary', 'style_secondary', 'bpm', 'beat_type', 'time_signature', 'full_midi_filename', 'full_audio_filename']


#### Let's look at a random entry!

In [17]:
ix =  int(np.random.random_sample()*dataset_size)
print("Sample Number: %d, Primary Style: %s, Secondary Style: %s" % (ix, 
                                                                     metadata["style_secondary"][ix], 
                                                                     metadata["style_primary"][ix])
     )

Sample Number: 5973, Primary Style: None, Secondary Style: rock


In [19]:
from bokeh.plotting import show
fig = train_set[ix].to_html_plot(filename="../pre_processing_tutorial/misc/temp.html",show_figure=False)
show(fig)


In [21]:
train_set[ix].save_audio(filename="../pre_processing_tutorial/misc/temp.wav", sf_path="../hvo_sequence/soundfonts/Standard_Drum_Kit.sf2")

array([0.        , 0.        , 0.        , ..., 0.00085397, 0.        ,
       0.        ])